In [1]:
import tensorflow as tf

# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

2024-10-24 20:12:58.275856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 20:12:58.287706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 20:12:58.300855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 20:12:58.304998: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 20:12:58.314796: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from modules.pinn import PINNModel, model1, optm1
from modules.data_gen import DataGenerator, real_u1, get_data
from modules.accuracy import AccuracyCalc

I0000 00:00:1729789979.581463   35893 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729789979.581667   35893 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729789979.581767   35893 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729789979.632284   35893 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
EPOCHS = 300
koefs = [1, 5, 100]
x = (0, 2)
y = (0, 2)

model = PINNModel(
    model1(), optm1(), "mse")
dg = DataGenerator(x, y, model.predict, real_u1)
acc = AccuracyCalc(dg, (70, 70))

I0000 00:00:1729789980.321629   36033 service.cc:146] XLA service 0x7f3d200053e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729789980.321656   36033 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-24 20:13:00.343351: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


137/154 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

I0000 00:00:1729789980.886507   36033 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [4]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for koef in koefs:
    model = PINNModel(
        model1(), optm1(), "mae")
    model.fit(koef, *dataset, EPOCHS)
    
    dg.update_predict(model.predict)
    acc.update_predicitons((70, 70))
    print(acc.mse(), acc.maxe())
    print(model.loss(koef, *dataset))
    print()

epoch: 0 loss: 103.641739
epoch: 299 loss: 0.0594228767
Time past 5.156548500061035

154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.006352171212989005 0.294623464345932
0.059147626

epoch: 0 loss: 98.9074097
epoch: 299 loss: 0.0865757093
Time past 4.1723737716674805

154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.0027141394739954123 0.32789260149002075
0.08633679

epoch: 0 loss: 104.529823
epoch: 299 loss: 0.0833093449
Time past 4.060110569000244

154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
2.2762324397127074e-05 0.03896741569042206
0.082987

